# libraries & global vars

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

# io orgtree & org codes

In [2]:
path_org_str = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 1 cleaning/cleaning step 5 - orgtree/"

In [3]:
filein_org_tree = "orgtree1_1.xlsx"

In [41]:
fileout_org_tree = "orgtree1_3"

# import newly parsed unique & multi strings

In [6]:
# read in orgtree db
orgtree_party = pd.read_excel(path_org_str+filein_org_tree,dtype="str",sheet_name="노동당")
orgtree_outside = pd.read_excel(path_org_str+filein_org_tree,dtype="str",sheet_name="당외곽")
orgtree_govt = pd.read_excel(path_org_str+filein_org_tree,dtype="str",sheet_name="정권기관")
orgtree_army = pd.read_excel(path_org_str+filein_org_tree,dtype="str",sheet_name="인민군")

In [7]:
orgtree_party.shape

(149, 10)

In [8]:
orgtree_govt.shape

(582, 10)

In [9]:
orgtree_army.shape

(289, 10)

In [10]:
orgtree_outside.shape

(391, 10)

In [11]:
orgtree_party.columns = orgtree_govt.columns

In [12]:
orgtree_army.columns = orgtree_govt.columns

In [13]:
orgtree_outside.columns = orgtree_govt.columns

In [14]:
orgtree_party.columns

Index(['ID', 'Org1', 'Org2', 'Org3', 'Org4', 'Org5', 'LinkToCurrent',
       'StatusUncertain', 'org1 error', '비고'],
      dtype='object')

In [15]:
orgtree_inside = pd.concat([orgtree_party,orgtree_army,orgtree_party,orgtree_govt])

In [16]:
orgtree_inside["InsideGov"] = True

In [17]:
orgtree_outside["InsideGov"] = False

In [18]:
orgtree = pd.concat([orgtree_inside,orgtree_outside])

In [19]:
orgtree.columns = ['ID', 'PrimaryInstitution', 'Org2', 'Org3', 'Org4', 'Org5', 'LinkToCurrent',
       'StatusUncertain', 'org1 error', '비고', 'InsideGov']

In [20]:
orgtree["OrgName"] = orgtree[["Org2","Org3","Org4","Org5"]].apply(lambda row: "_".join([str(item) for item in [row["Org2"],row["Org3"],row["Org4"],row["Org5"]] if item is not np.nan]),axis=1)

In [21]:
orgtree["ImmediateSuperiorOrg"] = orgtree["OrgName"].apply(lambda row: "_".join(row.split("_")[:-1]))

In [22]:
orgtree.loc[orgtree.LinkToCurrent=="NAN"] = np.nan

In [23]:
select_columns = ["ID","InsideGov",
                  "PrimaryInstitution","OrgName","ImmediateSuperiorOrg",
                  "LinkToCurrent","StatusUncertain","org1 error","비고"]
orgtree_concise = orgtree[select_columns]

# remove StatusUncertain and org1 error

In [ ]:
orgtree = orgtree_concise

In [37]:
# confirm no overlap between the two variables
orgtree[~orgtree["org1 error"].isna() & ~orgtree["StatusUncertain"].isna()]

,ID,PrimaryInstitution,Org2,Org3,Org4,Org5,LinkToCurrent,StatusUncertain,org1 error,비고,InsideGov,OrgName,ImmediateSuperiorOrg


In [38]:
orgtree_statusuncertain = orgtree[~orgtree["StatusUncertain"].isna()]
orgtree_statusuncertain.shape

(128, 13)

In [39]:
orgtree_org1error = orgtree[~orgtree["org1 error"].isna()]
orgtree_org1error.shape

(65, 13)

In [40]:
orgtree_no_errors = orgtree[orgtree["org1 error"].isna() & orgtree["StatusUncertain"].isna()]
orgtree_no_errors.shape

(1367, 13)

In [ ]:
# export rows with non-null StatusUncertain & org1 error
suffix
orgtree_statusuncertain.to_excel(path_org_str+fileout_org_tree+suffix)
orgtree_org1error.to_excel()

# replace LinkToCurrent with PrimaryInstitution and OrgName

In [566]:
orgtree = orgtree_concise

In [567]:
orgid = orgtree[["ID","PrimaryInstitution","OrgName"]]

In [568]:
orgtree_linked = orgtree.merge(orgid,how="left",left_on="LinkToCurrent",right_on="ID",suffixes=("","_LinkToCurrent"),indicator=True)

In [569]:
# all LinkToCurrent merged - excluding NaN and Current
orgtree_linked[((~orgtree_linked.LinkToCurrent.isna()) & (orgtree_linked.LinkToCurrent != "Current")) & (orgtree_linked["_merge"]=="left_only")]

,ID,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,LinkToCurrent,StatusUncertain,org1 error,비고,ID_LinkToCurrent,PrimaryInstitution_LinkToCurrent,OrgName_LinkToCurrent,_merge


In [570]:
# code PrimaryInstitution_LinkToCurrent and OrgName_LinkToCurrent as Current if LinkToCurrent == LinkToCurrent
orgtree_linked.loc[orgtree_linked.LinkToCurrent=="Current","PrimaryInstitution_LinkToCurrent"] = "Current"
orgtree_linked.loc[orgtree_linked.LinkToCurrent=="Current","OrgName_LinkToCurrent"] = "Current"

In [571]:
# drop LinkToCurrent, ID_LinkToCurrent, _merge
orgtree_linked.drop(columns=["LinkToCurrent","ID_LinkToCurrent","_merge"],inplace=True)

In [572]:
# rename PrimaryInstitution_LinkToCurrent --> LinkToCurrent_PrimaryInstitution, LinkToCurrent_OrgName
orgtree_linked.columns = ["ID",'InsideGov', 'PrimaryInstitution', 'OrgName', 'ImmediateSuperiorOrg',
       'StatusUncertain', 'org1 error', '비고',
       'LinkToCurrent_PrimaryInstitution', 'LinkToCurrent_OrgName']

In [573]:
orgtree_linked[0:5]

,ID,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,StatusUncertain,org1 error,비고,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName
0,1,True,당중앙군사위원회,,,NaN,NaN,NaN,Current,Current
1,2,True,당중앙검사위원회,,,NaN,NaN,NaN,Current,Current
2,3,True,당중앙위원회,정치국,,NaN,NaN,NaN,Current,Current
3,4,True,당중앙위원회,비서국,,NaN,NaN,NaN,Current,Current
4,5,True,당중앙위원회,조직지도부,,NaN,NaN,NaN,Current,Current


# group by unique InsideGov, PrimaryInstitution, OrgName
- delete ID
- calculate max StatusUncertain, org1 error
- list 비고

In [574]:
orgtree = orgtree_linked

In [575]:
orgtree[0:10]

,ID,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,StatusUncertain,org1 error,비고,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName
0,1,True,당중앙군사위원회,,,NaN,NaN,NaN,Current,Current
1,2,True,당중앙검사위원회,,,NaN,NaN,NaN,Current,Current
2,3,True,당중앙위원회,정치국,,NaN,NaN,NaN,Current,Current
3,4,True,당중앙위원회,비서국,,NaN,NaN,NaN,Current,Current
4,5,True,당중앙위원회,조직지도부,,NaN,NaN,NaN,Current,Current
5,6,True,당중앙위원회,선전선동부,,NaN,NaN,NaN,Current,Current
6,7,True,당중앙위원회,간부부,,NaN,NaN,NaN,Current,Current
7,8,True,당중앙위원회,경공업부,,NaN,NaN,NaN,Current,Current
8,9,True,당중앙위원회,경제부,,NaN,NaN,NaN,Current,Current
9,10,True,당중앙위원회,과학교육부,,NaN,NaN,NaN,Current,Current


In [576]:
orgtree.loc[~orgtree.StatusUncertain.isna(),"StatusUncertain"] = True
orgtree.loc[orgtree.StatusUncertain.isna(),"StatusUncertain"] = False

In [577]:
orgtree.loc[~orgtree["org1 error"].isna(),"org1 error"] = True
orgtree.loc[orgtree["org1 error"].isna(),"org1 error"] = False

In [578]:
orgtree[0:10]

,ID,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,StatusUncertain,org1 error,비고,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName
0,1,True,당중앙군사위원회,,,False,False,NaN,Current,Current
1,2,True,당중앙검사위원회,,,False,False,NaN,Current,Current
2,3,True,당중앙위원회,정치국,,False,False,NaN,Current,Current
3,4,True,당중앙위원회,비서국,,False,False,NaN,Current,Current
4,5,True,당중앙위원회,조직지도부,,False,False,NaN,Current,Current
5,6,True,당중앙위원회,선전선동부,,False,False,NaN,Current,Current
6,7,True,당중앙위원회,간부부,,False,False,NaN,Current,Current
7,8,True,당중앙위원회,경공업부,,False,False,NaN,Current,Current
8,9,True,당중앙위원회,경제부,,False,False,NaN,Current,Current
9,10,True,당중앙위원회,과학교육부,,False,False,NaN,Current,Current


In [579]:
# make sure that InsideGov & ImmediateSuperiorOrg are unique

In [580]:
# verify InsideGov is unique and not duplicated
orgtree_counts = orgtree.groupby(["PrimaryInstitution","OrgName"])["InsideGov"].apply(lambda x:len(set(x))).reset_index()
orgtree_counts[orgtree_counts.InsideGov > 1]

,PrimaryInstitution,OrgName,InsideGov


In [581]:
# verify ImmediateSuperiorOrg is unique and not duplicated
orgtree_counts = orgtree.groupby(["PrimaryInstitution","OrgName"])["ImmediateSuperiorOrg"].apply(lambda x:len(set(x))).reset_index()
orgtree_counts[orgtree_counts.ImmediateSuperiorOrg > 1]

,PrimaryInstitution,OrgName,ImmediateSuperiorOrg


In [582]:
# apply_dict = {"StatusUncertain":max,"org1 error":max,"비고":max}
orgtree_merge = orgtree.groupby(["InsideGov","PrimaryInstitution","OrgName","ImmediateSuperiorOrg","LinkToCurrent_PrimaryInstitution","LinkToCurrent_OrgName"])["StatusUncertain","org1 error","비고"].apply(max).reset_index()

C:\Users\seoul\AppData\Local\Temp\ipykernel_15596\2147953807.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  orgtree_merge = orgtree.groupby(["InsideGov","PrimaryInstitution","OrgName","ImmediateSuperiorOrg","LinkToCurrent_PrimaryInstitution","LinkToCurrent_OrgName"])["StatusUncertain","org1 error","비고"].apply(max).reset_index()
C:\Users\seoul\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:40: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial, where)


In [583]:
orgtree_merge[0:5]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,StatusUncertain,org1 error,비고
0,False,6.15공동선언실천 \n남북공동위원회 \n북측위원회,교직원분과,,Current,Current,False,False,NaN
1,False,6.15공동선언실천 \n남북공동위원회 \n북측위원회,노동자분과,,Current,Current,False,False,NaN
2,False,6.15공동선언실천 \n남북공동위원회 \n북측위원회,농업근로자분과,,Current,Current,False,False,NaN
3,False,6.15공동선언실천 \n남북공동위원회 \n북측위원회,문학예술분과,,Current,Current,False,False,NaN
4,False,6.15공동선언실천 \n남북공동위원회 \n북측위원회,보건분과,,Current,Current,False,False,NaN


# orgtree_concise - recursively add ImmedSuperiorOrgs, drop duplicates, sort

In [584]:
orgtree = orgtree_merge

In [585]:
orgtree[0:2]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,StatusUncertain,org1 error,비고
0,False,6.15공동선언실천 \n남북공동위원회 \n북측위원회,교직원분과,,Current,Current,False,False,NaN
1,False,6.15공동선언실천 \n남북공동위원회 \n북측위원회,노동자분과,,Current,Current,False,False,NaN


In [586]:
orgtree_2order = orgtree[orgtree.OrgName.str.contains("_")]
orgtree_2order.OrgName = orgtree_2order.OrgName.apply(lambda x: "_".join(x.split("_")[:-1]))
orgtree_2order.shape

C:\Users\seoul\AppData\Local\Temp\ipykernel_15596\1535604282.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orgtree_2order.OrgName = orgtree_2order.OrgName.apply(lambda x: "_".join(x.split("_")[:-1]))


(610, 9)

In [587]:
orgtree_3order = orgtree_2order[orgtree_2order.OrgName.str.contains("_")]
orgtree_3order.OrgName = orgtree_3order.OrgName.apply(lambda x: "_".join(x.split("_")[:-1]))
orgtree_3order.shape

C:\Users\seoul\AppData\Local\Temp\ipykernel_15596\1266295804.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orgtree_3order.OrgName = orgtree_3order.OrgName.apply(lambda x: "_".join(x.split("_")[:-1]))


(271, 9)

In [588]:
orgtree_4order = orgtree_3order[orgtree_3order.OrgName.str.contains("_")]
orgtree_4order.OrgName = orgtree_4order.OrgName.apply(lambda x: "_".join(x.split("_")[:-1]))
orgtree_4order.shape

C:\Users\seoul\AppData\Local\Temp\ipykernel_15596\1114847744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orgtree_4order.OrgName = orgtree_4order.OrgName.apply(lambda x: "_".join(x.split("_")[:-1]))


(38, 9)

In [589]:
orgtree_5order = orgtree_4order[orgtree_4order.OrgName.str.contains("_")]
orgtree_5order.shape

(0, 9)

In [590]:
# concatenate 1,2,3,4 order
orgtree = pd.concat([orgtree,orgtree_2order,orgtree_3order,orgtree_4order])
orgtree.shape

(2075, 9)

In [591]:
orgtree.loc[orgtree.OrgName.str.contains("_"),"ImmediateSuperiorOrg"] = orgtree.loc[orgtree.OrgName.str.contains("_"),"OrgName"].apply(lambda x: "_".join(x.split("_")[:-1]))

In [592]:
# drop duplicates
orgtree.drop_duplicates(inplace=True)
orgtree.shape

(1450, 9)

In [596]:
# sort
orgtree.sort_values(["InsideGov","PrimaryInstitution","OrgName"],ascending=False,inplace=True,ignore_index=True)

In [598]:
orgtree.shape

(1450, 9)

In [599]:
orgtree[0:100]

,InsideGov,PrimaryInstitution,OrgName,ImmediateSuperiorOrg,LinkToCurrent_PrimaryInstitution,LinkToCurrent_OrgName,StatusUncertain,org1 error,비고
0,True,최고재판소,,,최고인민회의,중앙재판소,False,False,NaN
1,True,최고인민회의,중앙재판소,,Current,Current,False,False,NaN
2,True,최고인민회의,중앙검찰소,,Current,Current,False,False,NaN
3,True,최고인민회의,외교위원회,,Current,Current,False,False,NaN
4,True,최고인민회의,외교위원회,,최고인민회의,외교위원회,False,False,NaN
5,True,최고인민회의,외교위,,최고인민회의,외교위원회,False,False,NaN
6,True,최고인민회의,외교분과위원회,,최고인민회의,외교위원회,False,False,NaN
7,True,최고인민회의,예산위원회,,Current,Current,False,False,NaN
8,True,최고인민회의,예산위원회,,최고인민회의,예산위원회,False,False,NaN
9,True,최고인민회의,예산심의위원회,,최고인민회의,예산위원회,False,False,NaN


# export orgtree_concise

In [600]:
orgtree.to_excel(path_org_str+fileout_org_tree_concise,index=False)